# Keyword Detection on Websites
Source: https://platform.stratascratch.com/data-projects/keyword-detection-websites


In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import chardet
from transformers import pipeline

### (1) Load and examine data

In [2]:
traindt = pd.read_csv("./datasets/train.csv")
testdt = pd.read_csv("./datasets/test.csv")

In [3]:
htmldt = []

for i in range(147):
    with open(f"./datasets/htmls/{i}.html", "rb") as f:
        initread = chardet.detect(f.read())
        encoding = initread["encoding"]

    with open(f"./datasets/htmls/{i}.html", "r", encoding=encoding) as f:
        html = f.read()

    soup = BeautifulSoup(html, "html.parser")
    body = soup.get_text(" ", strip=True)
    htmldt.append({"id": i, "html": body})

htmldt = pd.DataFrame(htmldt)

c:\Users\Jerry_Dell_7060\AppData\Local\Programs\Python\Python312\Lib\html\parser.py:171: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  k = self.parse_starttag(i)


In [4]:
htmldt

,id,html
0,0,"Bauchspeicheldrüse | Klinik für Allgemein-, Vi..."
1,1,Elbe-Elster Klinikum - Chirurgie Finsterwalde ...
2,2,Chirurgie der Bauchspeicheldrüse (Pankreaschir...
3,3,Onkologisches Zentrum - Klinikum Bayreuth Aktu...
4,4,Zentrum - Sozialpädiatrisches Zentrum - Städti...
...,...,...
142,142,Tumorboard des Lungenkrebszentrums Möglicherwe...
143,143,Oberarzt (m/w/d) für die HNO-Klinik A- A A+ No...
144,144,Für Ärzte | Vivantes JavaScript scheint in Ihr...
145,145,"Innere Medizin – Hämatologie, Onkologie und Pa..."


### (2) LLM based Classification

In [5]:
pipe = pipeline("question-answering", model="deepset/xlm-roberta-large-squad2")

Some weights of the model checkpoint at deepset/xlm-roberta-large-squad2 were not used when initializing XLMRobertaForQuestionAnswering: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu


In [10]:
QA_Input = {
    "question": "How many persons like this movie?",
    "context": "Tommy hate this movie, but Jason does not agree with him.",
}
res = pipe(QA_Input)
res


{'score': 2.5765925215637253e-07,
 'start': 26,
 'end': 57,
 'answer': ' Jason does not agree with him.'}